# Langroid Implementation

langroid:
tokenizer: https://github.com/simonw/ttok/issues/8

In [ ]:
import langroid.language_models as lm
import langroid as lr

# example interactive session
llm_config = lm.OpenAIGPTConfig(
    chat_model="ollama/llama2:latest",
    chat_context_length=16_000, # adjust based on model
)
agent_config = lr.ChatAgentConfig(
    llm=llm_config,
    system_message="You are helpful but concise",
)
agent = lr.ChatAgent(agent_config)
# directly invoke agent's llm_response method
# response = agent.llm_response("What is the capital of Russia?")
task = lr.Task(agent, interactive=True)
task.run() # for an interactive chat loop

In [ ]:
with open("private/hf-key", "r") as f:
    key = f.read().rstrip("\n")
import tokenizers
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token=key)

In [4]:
import langroid as lr
import langroid.language_models as lm

from typing import Dict
from langroid.utils.configuration import set_global
from langroid.agent.chat_document import ChatDocument

# set_global("openai-function-calling", True)

def _select_choice(questions: Dict[str,str])->str:
    return list(questions.keys())[0]

llm_config = lm.OpenAIGPTConfig(
    chat_model="ollama/llama3.2",
    chat_context_length =20_000
)

agent_config = lr.ChatAgentConfig(
    llm=llm_config,
    system_message="You are a 25 year old athletic woman. Please answer the following questions.",
)

questions = {
    "-8": "I don't know",
    "-7": "I prefer not to answer",
    "-1": "Appropriate skip",
    "1": "1 day a week",
    "2": "2 days a week",
    "3": "3 days a week",
    "4": "4 days a week",
    "5": "5 days a week",
    "6": "6 days a week",
    "7": "7 days a week"
}

from typing import List, Optional
import langroid as lr
from langroid.agent.tool_message import ToolMessage
import langroid.language_models as lm

llm_config = lm.OpenAIGPTConfig(
    chat_model = "ollama/llama2",
    chat_context_length=16_00
)

config = lr.ChatAgentConfig(
    llm=llm_config,
    system_message="You are a 25 year old athletic female. Please answer the following survey questions related to fitness."
)

class SurveyQuestionTool(lr.agent.ToolMessage):
    request: str = "answerQuestion"
    purpose: str = """
        To answer the provided survey question with an <answer_index>.
        Only respond with the index of the question.
        """
    answer_index: int

    # @classmethod
    # def examples(cls) -> List["ToolMessage"]:
    #     pass


class SurveyAgent(lr.ChatAgent):
    def __init__(self, config: lr.ChatAgentConfig):
        super().__init__(config)
        self.tool_expected = False
        self.answersIndices = []
        self.answers = []

    def answerQuestion(self, msg: SurveyQuestionTool):
        self.user_response(f"How often do you work out per week? {questions}")
        return msg.answer_index

    def llm_response(
        self, message: Optional[str | ChatDocument] = None
    ) -> Optional[ChatDocument]:
        self.tool_expected = True
        return super().llm_response(message)

    def user_response(
        self,
        msg: Optional[str | ChatDocument] = None,
    ) -> Optional[ChatDocument]:
        self.tool_expected = False
        return super().user_response(msg)

agent = SurveyAgent(config=config)
agent.enable_message(SurveyQuestionTool)


In [16]:
response = agent.llm_response("how are you")

1

Stats: N_MSG=8, TOKENS: in=290, out=2, max=8192, ctx=1600, COST: now=$0.0000, cumul=$0.0000, tot=$0.0000 (llama2)

In [18]:
agent.user_response()

Human (respond or q, x to exit current level, or hit enter to continue)
:

ChatDocument(content='tell us about you', metadata=ChatDocMetaData(source='User', source_content='context', is_chunk=False, id='a2ab2e99-1cd9-4f3b-81ce-c84c1177e024', window_ids=[], parent_id='', child_id='', agent_id='', msg_idx=-1, sender=<Entity.USER: 'User'>, oai_tool_id=None, tool_ids=[], block=None, sender_name='', recipient='', usage=None, cached=False, displayed=False, has_citation=False, status=None), reasoning='', content_any=None, oai_tool_calls=None, oai_tool_id2result=None, oai_tool_choice='auto', function_call=None, tool_messages=[], all_tool_messages=[], attachment=None)

In [12]:
response

ChatDocument(content='how are ou', metadata=ChatDocMetaData(source='User', source_content='context', is_chunk=False, id='05f8a52c-8d34-4ba7-a71a-85b33aba4116', window_ids=[], parent_id='', child_id='', agent_id='', msg_idx=-1, sender=<Entity.USER: 'User'>, oai_tool_id=None, tool_ids=[], block=None, sender_name='', recipient='', usage=None, cached=False, displayed=False, has_citation=False, status=None), reasoning='', content_any=None, oai_tool_calls=None, oai_tool_id2result=None, oai_tool_choice='auto', function_call=None, tool_messages=[], all_tool_messages=[], attachment=None)

In [ ]:

task = lr.Task(agent, interactive=True)
task.run()

In [3]:
agent.message_history

[LLMMessage(role=<Role.SYSTEM: 'system'>, name=None, tool_call_id=None, tool_id='', content='You are a 25 year old athletic female. Please answer the following survey questions related to fitness.\n\n=== GUIDELINES ON SOME TOOLS/FUNCTIONS USAGE ===\n            TOOL: answerQuestion:\n                        GUIDANCE: \n        IMPORTANT: When using this or any other tool/function, you MUST include a \n        `request` field and set it equal to the FUNCTION/TOOL NAME you intend to use.\n\n\n\n\n\n\n=== ALL AVAILABLE TOOLS and THEIR FORMAT INSTRUCTIONS ===\nYou have access to the following TOOLS to accomplish your task:\n\nTOOL: answerQuestion\n            PURPOSE: \n        To answer the provided survey question with an <answer_index>.\n        Only respond with the index of the question.\n\n            JSON FORMAT: {\n    "type": "object",\n    "properties": {\n        "request": {\n            "default": "answerQuestion",\n            "type": "string"\n        },\n        "answer_ind